# 6. Metamodel

A metamodel will be created as the model of models. We can obtain the probabilities given by each model and then, with these results, we can create a model using them as the new features and, in this case, the targets will be the label 0 or 1 depending on wether a condition has that MoA associated. In fact, we'll also have one model for each MoA or column.

(At this point we have already proved there's no point in taking the probabilities before forcing the controls to be 0.0 into account. So, we can get rid off them!)

MODELS TO BE USED:
- Random Forest Classifier: `1b_model_200_64.joblib` 
- Support Vector Classifier: `2a_model.joblib`
- Naive Bayes: `3a1_model.joblib` (with the imp features from the RFC, top 15)
- Logistic Regression: `4a1_model.joblib` (with the imp features from the RFC, top 15)

In [1]:
from sklearn.metrics import log_loss
from joblib import dump, load
import pandas as pd
import numpy as np

In [2]:
# Data to be used for the model:
xtrain = pd.read_csv("xtrain.csv", index_col=0)
ytrain = pd.read_csv("ytrain.csv", index_col=0)

# Data to be used for the predictions:
xtest = pd.read_csv("xtest.csv", index_col=0)
ytest = pd.read_csv("ytest.csv", index_col=0)

In [3]:
def list_logloss(ytest, proba_pred):
    log_loss_list = []
    for i in range(ytest.shape[1]):
        log_loss_list.append(log_loss(ytest.iloc[:, i], proba_pred.iloc[:, i], labels = [0, 1]))
    return log_loss_list

## a) RFC probabilities

In [4]:
RFC_model = load('output/1b_model_200_64.joblib')

#new dataframe for saving the predictions
proba_pred_RFC = pd.DataFrame(columns=ytest.columns)

name_col = ytest.columns.tolist()
for i in range(ytest.shape[1]): 
    proba_pred_RFC[name_col[i]] = RFC_model[i].predict_proba(xtrain)[:, 1]
    print(i, end=' ', flush=True)

/Users/belbordes/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/belbordes/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 

In [5]:
proba_pred_RFC["sig_id"]= xtrain.index.tolist()
proba_pred_RFC = proba_pred_RFC.set_index('sig_id')
proba_pred_RFC.loc[xtrain.cp_type == 1, :] = 0
proba_pred_RFC.head(5)

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
sig_id,,,,,,,,,,,,,,,,,,,,,
id_000644bb2,0.000052,0.002261,0.000154,0.006034,0.018302,0.005222,0.002925,0.003602,0.000153,0.012950,...,0.000000,0.000000,0.001549,0.000704,0.002174,0.000101,0.002232,0.000311,0.001226,0.001743
id_000779bfc,0.000191,0.000225,0.000172,0.007732,0.008358,0.000774,0.001099,0.002804,0.000055,0.007892,...,0.000187,0.000248,0.000209,0.000206,0.001549,0.000000,0.000707,0.001094,0.000432,0.001471
id_000a6266a,0.001285,0.000859,0.001869,0.010876,0.017281,0.004576,0.002748,0.005809,0.001605,0.020600,...,0.000054,0.001080,0.003089,0.010551,0.001480,0.000200,0.042972,0.000409,0.002561,0.002897
id_0015fd391,0.000159,0.001088,0.001446,0.007361,0.008430,0.001133,0.000811,0.001772,0.000519,0.006180,...,0.000000,0.004625,0.002334,0.168074,0.004690,0.000425,0.008898,0.001288,0.000669,0.000851
id_001626bd3,0.001015,0.001800,0.004536,0.015056,0.020370,0.007301,0.006812,0.008186,0.001040,0.029784,...,0.001055,0.002397,0.003628,0.013834,0.005307,0.000321,0.010340,0.004865,0.002320,0.002047


## b) SVC probabilities

In [6]:
SVC_model = load('output/2a_model.joblib') #list with the models for each target column

#new dataframe for saving the predictions
proba_pred_SVC = pd.DataFrame(columns=ytest.columns)

name_col = ytest.columns.tolist()
for i in range(ytest.shape[1]): 
    proba_pred_SVC[name_col[i]] = SVC_model[i].predict_proba(xtrain)[:, 1]
    print(i, end=' ', flush=True)

/Users/belbordes/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator SVC from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 

In [7]:
proba_pred_SVC["sig_id"]= xtrain.index.tolist()
proba_pred_SVC = proba_pred_SVC.set_index('sig_id')
proba_pred_SVC.loc[xtrain.cp_type == 1, :] = 0
proba_pred_SVC.head(5)

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
sig_id,,,,,,,,,,,,,,,,,,,,,
id_000644bb2,2.674565e-04,0.001048,0.000995,0.005900,0.018258,0.004052,0.002882,0.001362,0.000090,0.012619,...,0.000090,0.000012,0.001716,0.002081,0.001761,0.000177,0.000322,0.001253,0.000006,0.002579
id_000779bfc,9.670806e-05,0.000625,0.001038,0.010600,0.014467,0.004083,0.002052,0.003766,0.000035,0.003309,...,0.000257,0.000418,0.001898,0.002544,0.001993,0.000183,0.001706,0.001683,0.000574,0.002159
id_000a6266a,1.410812e-05,0.000766,0.001040,0.004229,0.018293,0.003214,0.002520,0.003568,0.000659,0.005150,...,0.000158,0.000478,0.000915,0.001263,0.001014,0.000395,0.020407,0.000691,0.000273,0.001726
id_0015fd391,9.965451e-07,0.000858,0.000993,0.007734,0.007578,0.002374,0.002375,0.002091,0.000002,0.000909,...,0.000136,0.004174,0.001341,0.018410,0.001972,0.000327,0.000050,0.000996,0.000002,0.000443
id_001626bd3,4.280354e-05,0.000794,0.001109,0.009555,0.010220,0.002744,0.002770,0.002512,0.000278,0.027426,...,0.000470,0.000167,0.000813,0.002790,0.001506,0.000303,0.002815,0.001555,0.000080,0.002082


## c) NB probabilities

In [8]:
NB_model = load('output/3a1_model.joblib')
imp_features = load('important_features_RFC_15.joblib')

proba_pred_NB = pd.DataFrame(columns=ytest.columns)

name_col = ytest.columns.tolist()
for i in range(ytest.shape[1]): 
    proba_pred_NB[name_col[i]] = NB_model[i].predict_proba(xtrain.iloc[:, imp_features[i].tolist()])[:, 1]
    print(i, end=' ', flush=True)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 

In [9]:
proba_pred_NB["sig_id"]= xtrain.index.tolist()
proba_pred_NB = proba_pred_NB.set_index('sig_id')
proba_pred_NB.loc[xtrain.cp_type == 1, :] = 0
proba_pred_NB.head(5)

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
sig_id,,,,,,,,,,,,,,,,,,,,,
id_000644bb2,6.085114e-04,0.456340,0.005417,1.816787e-02,9.651320e-01,0.393794,0.051739,4.511197e-01,1.593294e-06,6.741140e-02,...,2.123343e-04,0.000171,2.377669e-02,1.483583e-05,0.000417,9.193571e-05,0.028762,0.001596,4.982138e-06,2.700689e-01
id_000779bfc,1.623319e-12,0.029283,0.003072,2.240646e-01,8.652744e-01,0.339391,0.112679,1.233626e-01,2.409643e-04,8.509354e-01,...,2.991058e-05,0.007808,6.319533e-02,1.555344e-07,0.000830,1.045505e-04,0.030814,0.006839,9.165235e-03,1.384688e-02
id_000a6266a,2.335837e-53,0.079823,0.107604,3.206329e-03,1.521984e-01,0.256203,0.000381,1.385686e-02,4.910642e-04,8.421274e-01,...,2.314166e-12,0.000033,2.777754e-04,7.363143e-08,0.003615,1.656176e-06,0.008645,0.004145,1.456416e-07,2.727999e-03
id_0015fd391,4.586170e-24,0.000255,0.000347,4.061703e-11,7.208058e-34,0.005930,0.000009,1.321690e-08,1.167355e-37,1.995761e-12,...,1.311638e-17,0.003713,2.613815e-09,4.254340e-01,0.019688,5.504843e-09,0.000014,0.005542,4.975834e-20,1.120840e-12
id_001626bd3,8.098875e-12,0.004615,0.045910,4.912132e-03,1.937106e-01,0.022405,0.159328,5.472007e-01,8.647079e-07,3.858927e-01,...,3.211983e-09,0.000407,1.724880e-03,8.578195e-06,0.003474,4.547450e-04,0.073191,0.127168,6.847485e-04,1.856768e-02


## d) LR probabilities

In [10]:
LR_model = load('output/4a1_model.joblib')
imp_features = load('important_features_RFC_15.joblib')

proba_pred_LR = pd.DataFrame(columns=ytest.columns)

name_col = ytest.columns.tolist()
for i in range(ytest.shape[1]): 
    proba_pred_LR[name_col[i]] = LR_model[i].predict_proba(xtrain.iloc[:, imp_features[i].tolist()])[:, 1]
    print(i, end=' ', flush=True)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 

In [11]:
proba_pred_LR["sig_id"]= xtrain.index.tolist()
proba_pred_LR = proba_pred_LR.set_index('sig_id')
proba_pred_LR.loc[xtrain.cp_type == 1, :] = 0
proba_pred_LR.head(5)

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
sig_id,,,,,,,,,,,,,,,,,,,,,
id_000644bb2,1.109856e-04,0.003396,0.001813,0.006501,0.051071,0.004706,0.000977,0.004070,4.607191e-06,0.006953,...,1.584836e-04,0.000014,0.001319,0.009592,0.000969,0.000360,0.004299,0.000240,5.958408e-05,0.004744
id_000779bfc,3.295886e-05,0.002191,0.000592,0.011266,0.008747,0.001460,0.001140,0.002816,4.882609e-05,0.021310,...,9.126641e-05,0.000253,0.001600,0.002606,0.001757,0.000083,0.002917,0.000813,5.319878e-04,0.001684
id_000a6266a,1.454779e-05,0.002867,0.013971,0.001284,0.010914,0.002120,0.000854,0.003204,3.754485e-05,0.028398,...,2.376890e-06,0.000391,0.000764,0.001911,0.003468,0.000032,0.028466,0.000426,1.748122e-05,0.000130
id_0015fd391,1.083275e-07,0.000719,0.000140,0.000601,0.000709,0.001417,0.000525,0.006560,9.457401e-14,0.000787,...,1.868039e-07,0.001977,0.000326,0.028163,0.017190,0.000011,0.000076,0.000661,1.211446e-07,0.000026
id_001626bd3,3.392684e-05,0.000835,0.006963,0.009277,0.015422,0.002484,0.001900,0.011153,1.206347e-05,0.019165,...,6.480296e-05,0.006965,0.000121,0.000919,0.005501,0.000072,0.004054,0.059541,7.838522e-04,0.002954


----------------
# Metamodel for each column

In this section we want to create a new model for each column (for each MoA). Instead of working with the features, we'll be working with the predictions of the different models as the input.

Given the bad results we obtained from the NB algorithm, we have decided to get rid off them! It's reasonable to do it since the sklearn webpage warning was about the inaccuracy of the probabilities of this model.

### Datasets

In [13]:
#CREATING THE DATAFRAMES THAT WILL BE USED TO TRAIN THE METAMODEL
names_MOA = ytrain.columns.tolist()
dict_models_df = {} #it will include all the dataframes for each MoA
for moa in names_MOA:
    dict_models_df[moa] = pd.DataFrame()
    
    dict_models_df[moa]['SVC'] = proba_pred_SVC[moa]
    dict_models_df[moa]['RFC'] = proba_pred_RFC[moa]
    dict_models_df[moa]['LR'] = proba_pred_LR[moa]
    #dict_models_df[moa]['NB'] = proba_pred_NB[moa]
    
    dict_models_df[moa]['label'] = ytrain[moa]

### Models

In [14]:
models_list = []

index = 0
from sklearn.ensemble import RandomForestClassifier
for moa in names_MOA:
    models_list.append(RandomForestClassifier())
    models_list[index].fit(dict_models_df[moa].iloc[:, :-1], dict_models_df[moa].iloc[:, -1])
    print(index, end=' ', flush=True)
    index+=1
len(models_list)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 

206

In [15]:
dump(models_list, 'output/6_model.joblib')

['output/6_model.joblib']

_____

## Metamodel used for the test set

Now we have to compute the probabilities obtained by applying all the 4 models to the test dataset.

### a) RFC

In [16]:
#new dataframe for saving the predictions
proba_pred_RFC = pd.DataFrame(columns=ytest.columns)

name_col = ytest.columns.tolist()
for i in range(ytest.shape[1]): 
    proba_pred_RFC[name_col[i]] = RFC_model[i].predict_proba(xtest)[:, 1]
    print(i, end=' ', flush=True)
    
proba_pred_RFC["sig_id"]= xtest.index.tolist()
proba_pred_RFC = proba_pred_RFC.set_index('sig_id')
proba_pred_RFC.loc[xtest.cp_type == 1, :] = 0

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 

### b) SVC

In [17]:
#new dataframe for saving the predictions
proba_pred_SVC = pd.DataFrame(columns=ytest.columns)

name_col = ytest.columns.tolist()
for i in range(ytest.shape[1]): 
    proba_pred_SVC[name_col[i]] = SVC_model[i].predict_proba(xtest)[:, 1]
    print(i, end=' ', flush=True)
    
proba_pred_SVC["sig_id"]= xtest.index.tolist()
proba_pred_SVC = proba_pred_SVC.set_index('sig_id')
proba_pred_SVC.loc[xtest.cp_type == 1, :] = 0

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 

### c) NB

In [56]:
proba_pred_NB = pd.DataFrame(columns=ytest.columns)

name_col = ytest.columns.tolist()
for i in range(ytest.shape[1]): 
    proba_pred_NB[name_col[i]] = NB_model[i].predict_proba(xtest.iloc[:, imp_features[i].tolist()])[:, 1]
    print(i, end=' ', flush=True)
    
proba_pred_NB["sig_id"]= xtest.index.tolist()
proba_pred_NB = proba_pred_NB.set_index('sig_id')
proba_pred_NB.loc[xtest.cp_type == 1, :] = 0

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 

### d) LR

In [18]:
proba_pred_LR = pd.DataFrame(columns=ytest.columns)

name_col = ytest.columns.tolist()
for i in range(ytest.shape[1]): 
    proba_pred_LR[name_col[i]] = LR_model[i].predict_proba(xtest.iloc[:, imp_features[i].tolist()])[:, 1]
    print(i, end=' ', flush=True)
    
proba_pred_LR["sig_id"]= xtest.index.tolist()
proba_pred_LR = proba_pred_LR.set_index('sig_id')
proba_pred_LR.loc[xtest.cp_type == 1, :] = 0

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 

___
Time to use the model with our data!

In [20]:
#CREATING THE DATAFRAMES THAT WILL BE USED TO TEST THE METAMODEL
names_MOA = ytrain.columns.tolist()
dict_models_df = {} #it will include all the datframes for each MoA
for moa in names_MOA:
    dict_models_df[moa] = pd.DataFrame()
    
    dict_models_df[moa]['SVC'] = proba_pred_SVC[moa]
    dict_models_df[moa]['RFC'] = proba_pred_RFC[moa]
    dict_models_df[moa]['LR'] = proba_pred_LR[moa]
    #dict_models_df[moa]['NB'] = proba_pred_NB[moa]
    
    #dict_models_df[moa]['label'] = ytrain[moa]

In [22]:
dict_models_df[names_MOA[0]]

,SVC,RFC,LR
sig_id,,,
id_0031dd989,0.000677,0.002070,1.905276e-04
id_0077e36b9,0.000414,0.000282,1.849728e-04
id_0089590d4,0.001359,0.000308,7.443384e-05
id_008c40337,0.001481,0.000109,9.240591e-05
id_00ac7c837,0.000194,0.001458,2.068993e-03
...,...,...,...
id_ff7b158fa,0.000652,0.000850,3.685067e-04
id_ffa08c24c,0.000006,0.000679,4.369620e-08
id_ffa27d492,0.000177,0.000187,1.769355e-04


In [23]:
proba_pred_metamodel_test = pd.DataFrame(columns=ytest.columns)

name_col = ytest.columns.tolist()
for i in range(ytest.shape[1]): 
    proba_pred_metamodel_test[name_col[i]] = models_list[i].predict_proba(dict_models_df[name_col[i]])[:, 1]
    print(i, end=' ', flush=True)
    
proba_pred_metamodel_test["sig_id"]= xtest.index.tolist()
proba_pred_metamodel_test = proba_pred_metamodel_test.set_index('sig_id')
proba_pred_metamodel_test.loc[xtest.cp_type == 1, :] = 0

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 

In [24]:
proba_pred_metamodel_test

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
sig_id,,,,,,,,,,,,,,,,,,,,,
id_0031dd989,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
id_0077e36b9,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
id_0089590d4,0.0,0.0,0.0,0.0,0.18,0.0,0.0,0.32,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
id_008c40337,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
id_00ac7c837,0.0,0.0,0.0,0.0,0.27,0.0,0.0,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
id_ff7b158fa,0.0,0.0,0.0,0.0,0.18,0.0,0.0,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
id_ffa08c24c,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0
id_ffa27d492,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


In [25]:
proba_pred_metamodel_test.to_csv('output/6_probas.csv')